In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus
op = Optimus()

In [4]:
# make some test data
columns = ['todo_id']
vals = [
    (1, ),
    (2, ),
    (3, ),
    (4, ),
    (5, )

]

# create DataFrame
df = op.spark.createDataFrame(vals, columns).repartition(1).cache()

In [5]:
df.table()

todo_id 1 (bigint)
1
2
3
4
5


In [6]:
df.save.rabbit_mq("amqp://localhost:5672/","exchange_name", "example-queue","BOB"  )

In [7]:
df.save.mongo("localhost",27017, "db","BOB")

In [8]:
from pyspark.sql import functions as F
  
df_url = df.cols.nest(("https://jsonplaceholder.typicode.com/todos/",F.col("todo_id")),"url")
df_url.table()

todo_id 1 (bigint),url 2 (string)
1,https://jsonplaceholder.typicode.com/todos/1
2,https://jsonplaceholder.typicode.com/todos/2
3,https://jsonplaceholder.typicode.com/todos/3
4,https://jsonplaceholder.typicode.com/todos/4
5,https://jsonplaceholder.typicode.com/todos/5


In [42]:
from optimus.enricher import Enricher

e = Enricher("localhost",27017, "db","BOB")
e.send(df_url)

In [44]:
def func(response):
    return response["title"]

e.run(func_response= func)

In [45]:
e.save_to_csv("jazz.csv")

In [28]:
e.flush()

Removed 10 documents


In [38]:
e.show_keys()

['url', 'result', 'todo_id', '_id']

In [58]:
e.head("BOB",5)

Total documents:15
{'_id': ObjectId('5b940c37e254121740b5d19f'), 'todo_id': 1, 'url': 'https://jsonplaceholder.typicode.com/todos/1', 'result': 'delectus aut autem'}
{'_id': ObjectId('5b940c37e254121740b5d1a0'), 'todo_id': 2, 'url': 'https://jsonplaceholder.typicode.com/todos/2', 'result': 'quis ut nam facilis et officia qui'}
{'_id': ObjectId('5b940c37e254121740b5d1a1'), 'todo_id': 3, 'url': 'https://jsonplaceholder.typicode.com/todos/3', 'result': 'fugiat veniam minus'}
{'_id': ObjectId('5b940c37e254121740b5d1a2'), 'todo_id': 4, 'url': 'https://jsonplaceholder.typicode.com/todos/4', 'result': 'et porro tempora'}
{'_id': ObjectId('5b940c37e254121740b5d1a3'), 'todo_id': 5, 'url': 'https://jsonplaceholder.typicode.com/todos/5', 'result': 'laboriosam mollitia et enim quasi adipisci quia provident illum'}
